<a href="https://colab.research.google.com/github/MelissaTESSA/Development-of-a-word-embedding-model-for-ARABIZI-using-fasttext/blob/main/Code_Preprocessing_for_ARABIZI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
nltk.download('stopwords')

In [ ]:
pip install XlsxWriter

In [ ]:
pip install unidecode

In [ ]:
pip install fasttext

In [ ]:
pip install PyArabic

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
import csv
import unicodedata as ud #new added

import sklearn
import pickle

from tensorflow.keras.models import load_model
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.metrics import Accuracy,Recall,Precision
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [ ]:
#new added
import pyarabic.araby as araby
import pyarabic.number as number
from pyarabic.araby import strip_harakat
from pyarabic.araby import strip_tashkeel
from pyarabic.araby import strip_lastharaka
from pyarabic.araby import strip_tatweel
from pyarabic.araby import strip_shadda

import xlsxwriter
import unidecode

In [ ]:
UP = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
LOW = "abcdefghijklmnopqrstuvwxyz"
LATIN = "Ààéêèç"
SPACE = '\t\n\r\v\f'
fr_stopwords=stopwords.words('french')
en_stopwords=stopwords.words('english')

LETTERS =  'ہہچپڨابتةثجحخدذرزسشصضطظعغفقكلمنهويءآأؤإؤئئىىئ' #added characters

#all_latin_chars = UP + LOW + LATIN

punctuations = string.punctuation

punctuations_list =  punctuations + SPACE

french_punctuations_list= punctuations_list.replace("'","")

#Full List of emogies to be removed
emoji_pattern = re.compile("["
                       #u"\U0001F600-\U0001F64F"  # emoticons to keep 
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       u"\U0001f932"
                       u"\u200f"
                       u"\U0001F914"
                       u"\U0001F923"
                       u"\u200D"
                       u"\u202c"
                       u"\u2069"
                       u"\u2066"
                       u"\U0001F926"
                       u"\U0001F917"
                       u"\U0001f928"
                       u"\t"
                       u"\u200e"
                       "]+", flags=re.UNICODE)

In [ ]:
#Remove small words 
def remove_minlen_word(text, threshold):
    return " ".join([w for w in text.split() if len(w) > threshold])

#lower text
def lower(text):
    return text.lower()

#Remove urls
def remove_url(text):
    text = re.sub('http[s]?://\S+', '', text, flags=re.MULTILINE)
    text = re.sub('ftp?://\S+', '', text, flags=re.MULTILINE)
    text = re.sub('www.\S+', '', text, flags=re.MULTILINE)
    return text


#remove tags @ and hashtags #
def remove_prefix(text, prefix):  # mainly used to remove hashtags and mensions
    purged=[]
    output=""
    for word in text.split():
        if not word.startswith(prefix):
            purged.append(word)
    return " ".join(purged)

# Remove punctuation
def remove_french_punctuations(text):
    translator = str.maketrans('', '', french_punctuations_list)
    return text.translate(translator)

def remove_arabic_punct(text): #new added
    return ''.join(c for c in text if not ud.category(c).startswith('P'))

#remove french stop words, here we specified just Fr stop words, you can add other DZ word that you juge as stop words 
def remove_fr_stop_words(text):
    temp_text=[]
    for word in text.split(" "):
        if word not in fr_stopwords:
            temp_text.append(word)
    return ' '.join(temp_text)

# Remove repetting letters 
def remove_repeating_char(text):
    result=[]
    for word in text.split(" "):
        temp=re.sub(r'(.)\1+', r'\1', word)
        if len(temp)==1:
             temp=temp+temp
        result.append( temp)
    return " ".join(result)

# remove emogies 
def remove_emoji(text):
    return emoji_pattern.sub(r' ' , text)

# remove extra stop words 
def remove_extra_white_spaces(text):
    text= ' '.join(text.split())
    return text

# remove digits delimited with white space
def remove_digits(text):
    text= re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    return text

# specify a list of generic words to be removed if you want =W we can add here the arabizi stop words 
def remove_words_generic(text,list_words=[]):
    purged=[]
    for word in text.split(" "):
        if word not in list_words:
            purged.append(word)
    return " ".join(purged)

#remove all arabic characters from a text
def remove_arabic_chars(text):
    new_text = ""
    for char in text:
        if char not in LETTERS:
            new_text += char
    return new_text

#normalize arabic numbers
def normalize_arabic_numbers(text):
    text = re.sub("٠", "0", text)
    text = re.sub("١", "1", text)
    text = re.sub("٢", "2", text)
    text = re.sub("٣", "3", text)
    text = re.sub("٤", "4", text)
    text = re.sub("٥", "5", text)
    text = re.sub("٦", "6", text)
    text = re.sub("٧", "7", text)
    text = re.sub("٨", "8", text)
    text = re.sub("٩", "9", text)
    return text

# normalize arabizi characters
def normalize_arabic(text):
    text = re.sub("2", "a", text)
    text = re.sub("3", "aa", text)
    text = re.sub("7", "h", text)
    text = re.sub("9", "q", text)
    text = re.sub("8","gh", text)
    text = re.sub("5", "kh", text)
    text = re.sub("6", "t", text) #new added
    return text

In [ ]:
#this is the pipeline of preprocessing 
def my_preproc(data="",list_words=[]):
    text = remove_url(data)
    text= lower(text)
    text= remove_minlen_word(text,1)  #words containing one letter
    text=normalize_arabic_numbers(text) 
    text=remove_arabic_chars(text)
    text=remove_digits(text) 
    text=normalize_arabic(text)
    text=remove_prefix(text,"@")
    text=remove_prefix(text,"#")
    text=remove_words_generic(text, list_words)
    text=remove_french_punctuations(text) 
    text=remove_arabic_punct(text) #new added
    text=remove_fr_stop_words(text) 
    text=remove_repeating_char(text)
    text=remove_extra_white_spaces(text) 
    text= remove_emoji(text)
    return text

In [ ]:
text="rani 7aba nro7 l alg8ia 187"
my_preproc(data=text)

In [ ]:
#read csv file
name ="cevital"
df = pd.read_csv(name+".csv", error_bad_lines=False, engine = 'python')

#compose list of stopwords and names from files
with open('stopwords.txt','r') as f1:
  with open('prenoms.txt','r') as f2:
      l1 = f1.readlines()
      l2 = f2.readlines()
      lines = l1+l2
      for i in range(len(lines)):
        lines[i] = lines[i].split()[0].strip()
        lines[i] = lines[i].lower()

#preprocess the file
for i in range(len(df)):
  text = str(df.iloc[i,0]) 
  #new added : remove arabic characters and tashkeel
  text = strip_tashkeel(text)
  text = strip_harakat(text)
  text = strip_lastharaka(text)
  text = strip_shadda(text)
  text = strip_tatweel(text)
  #preprocess data
  df.iloc[i,0] = my_preproc(data=text, list_words=lines)

print("Original dataset :", len(df))
a = len(df)
#remove duplicates
df.drop_duplicates(inplace = True) 
print("New dataset :", len(df))
b= len(df)
print("percentage of kept data :", (b/a)*100)

rows = []
for i in range(len(df)):
  #remove empty cells
  if (str(df.iloc[i,0]).strip()!=""):
    rows.append(str(df.iloc[i,0]).strip())

print("Length of final list :",len(rows))

#write into excel file
workbook = xlsxwriter.Workbook(name+'.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write_column('A1', rows)
workbook.close()

print("Done")

********************************************************************************
Util functions
********************************************************************************


In [ ]:
#detect language
import fasttext
import pandas as pd
import xlsxwriter

PRETRAINED_MODEL_PATH = './lid.176.bin'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)

def predict_language(text):
  predictions = model.predict([text])
  return str(predictions[0])[12:14], predictions[1][0].item()

df = pd.read_excel(r'last.xlsx')
rows = []

#for stats
a = len(df)
b=0
for i in range(len(df)):
    if predict_language(str(df.iloc[i,0]).strip())[1]<0.45 :
        b+=1
        rows.append(str(df.iloc[i,0]).strip())
        print(str(df.iloc[i,0]).strip())

print("***********************************************************")
print(a)
print(b)
print("percentage:", (b/a)*100)

#write into excel file
workbook = xlsxwriter.Workbook('last_n.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write_column('A1', rows)
workbook.close()

In [ ]:
#remove arabic chars
import pandas as pd
import xlsxwriter
import unicodedata

def remove_accents(text):
    return ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')) 

chars= "abcdefghijklmnopqrstuvwxyz"
def contains_text(text):
  for c in text:
    if c in chars:
      return True
  return False

LETTERS =  'ییہہچپڨابتةثجحخدذرزسشصضطظعغفقكلمنهويءآأؤإؤئئىىئگـڤڤۦکی'
#remove all arabic characters from a text
def remove_arabic_chars(text):
    new_text = ""
    for char in text:
        if char not in LETTERS:
            new_text += char
    return new_text

df = pd.read_excel(r'01 dataset.xlsx')
rows = []

#for stats
a = len(df)
b=0
for i in range(len(df)):
    try:
        #remove first word
        text = str(df.iloc[i,0]).strip().split(" ", 1)[1]
        #remove accents (diacritics)
        text = remove_accents(text)
        #remove arabic letters
        text = remove_arabic_chars(text)
        #remove emojis and empty fields
        if text!="" and text.isnumeric()==False and contains_text(text):
            b+=1
            #rows.append(text.strip())
            rows.append(str(df.iloc[i,0]).strip()) 
            print(text)
    except:
        print("exception")
        #pass
    

print("***********************************************************")
print(a)
print(b)
print("percentage:", (b/a)*100)

#write into excel file
workbook = xlsxwriter.Workbook('01 dataset.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write_column('A1', rows)
workbook.close()

In [ ]:
#remove diacritics
import pandas as pd
import xlsxwriter
import re
from unidecode import unidecode
import unicodedata

def remove_accents(text):
    return ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')) 

chars= "abcdefghijklmnopqrstuvwxyz"
def contains_text(text):
  for c in text:
    if c in chars:
      return True
  return False

LETTERS =  'ہہچپڨابتةثجحخدذرزسشصضطظعغفقكلمنهويءآأؤإؤئئىىئگـڤڤ'
#remove all arabic characters from a text
def remove_arabic_chars(text):
    new_text = ""
    for char in text:
        if char not in LETTERS:
            new_text += char
    return new_text

df = pd.read_excel(r'01 dataset.xlsx')
rows = []

#for stats
a = len(df)
b=0
for i in range(len(df)):
    try:
        #text = unidecode.unidecode(str(df.iloc[i,0]).strip()) 
        text = remove_accents(str(df.iloc[i,0]).strip())
        if text!="" and text.isnumeric()==False and contains_text(text):
            b+=1
            rows.append(text.strip())
            print(text)
    except:
        print("exception")
    

print("***********************************************************")
print(a)
print(b)
print("percentage:", (b/a)*100)

#write into excel file
workbook = xlsxwriter.Workbook('01 dataset.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write_column('A1', rows)
workbook.close()

In [ ]:
#remove comments doesn't containing text (only emojis)
import pandas as pd
import xlsxwriter

chars= "abcdefghijklmnopqrstuvwxyz"
def contains_text(text):
  for c in text:
    if c in chars:
      return True
  return False

df = pd.read_excel(r'Melissa_dataset_V02.xlsx')
rows = []

#for stats
a = len(df)
b=0
for i in range(len(df)):
    if contains_text(str(df.iloc[i,0]).strip()) :
        b+=1
        rows.append(str(df.iloc[i,0]).strip())
        print(str(df.iloc[i,0]).strip())

print("***********************************************************")
print(a)
print(b)
print("percentage:", (b/a)*100)

#write into excel file
workbook = xlsxwriter.Workbook('02 dataset.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write_column('A1', rows)
workbook.close()

In [ ]:
#remove first word of each comment
import pandas as pd
import xlsxwriter

chars= "abcdefghijklmnopqrstuvwxyz"
def contains_text(text):
  for c in text:
    if c in chars:
      return True
  return False

df = pd.read_excel(r'02 dataset.xlsx')
rows = []

#for stats
a = len(df)
b=0
for i in range(len(df)):
    try:
        text = str(df.iloc[i,0]).strip().split(" ", 1)[1]
        if text!="" and text.isnumeric()==False and contains_text(text):
            b+=1
            rows.append(str(df.iloc[i,0]).strip())
            print(text)
    except:
        print("exception")
    

print("***********************************************************")
print(a)
print(b)
print("percentage:", (b/a)*100)

#write into excel file
workbook = xlsxwriter.Workbook('02 dataset.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write_column('A1', rows)
workbook.close()

In [ ]:
'''
import xlsxwriter
import unidecode


#read csv file
df = pd.read_csv("bimo.csv", error_bad_lines=False, engine = 'python')

#compose list of stopwords and names
with open('stopwords.txt','r') as f1:
  with open('prenoms.txt','r') as f2:
      l1 = f1.readlines()
      l2 = f2.readlines()
      lines = l1+l2
      for i in range(len(lines)):
        lines[i] = lines[i].split()[0].strip()
        lines[i] = lines[i].lower()

#preprocess the file
for i in range(len(df)):
  text = str(df.iloc[i,0])
  df.iloc[i,0] = my_preproc(data=text, list_words=lines)
  #df.iloc[i,0] = unidecode.unidecode(df.iloc[i,0])

print("Original dataset :", len(df))
a = len(df)
#remove duplicates
df.drop_duplicates(inplace = True) 
print("New dataset :", len(df))
b= len(df)
print("percentage of kept data :", (b/a)*100)

rows = []
for i in range(len(df)):
  #remove empty cells
  if (str(df.iloc[i,0]).strip()!=""):
    rows.append(str(df.iloc[i,0]).strip())

print("Length of final list :",len(rows))

#write into excel file
workbook = xlsxwriter.Workbook('db.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write_column('A1', rows)
workbook.close()

print("Done")'''

In [ ]:
'''
#write into excel file
workbook = xlsxwriter.Workbook('db.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write_column('A1', rows)
workbook.close()

print("Done")'''

'''
for i in range(len(df)):
  if (str(df.iloc[i,0])!=""):
    print(df.iloc[i,0])

with open('db.csv', 'w') as f:
  # create the csv writer
  writer = csv.writer(f)
  for row in rows:
    writer.writerow(row)
'''


#df.to_csv("db.csv", index=False)
#df.to_excel("db.xlsx", index=False)

In [ ]:
with open('stopwords.txt','r') as f1:
  with open('prenoms.txt','r') as f2:
      l1 = f1.readlines()
      l2 = f2.readlines()
      lines = l1+l2
      for i in range(len(lines)):
        lines[i] = lines[i].split()[0].strip()
        lines[i] = lines[i].lower()

In [ ]:
chars= "abcdefghijklmnopqrstuvwxyz"
def contains_text(text):
  for c in text:
    if c in chars:
      return True
  return False

In [ ]:
'''
+araby.HARAKAT+araby.LIGUATURES+araby.TANWIN+araby.TASHKEEL+araby.SHORTHARAKAT+araby.HAMZAT+araby.ALEFAT
+araby.WEAK+araby.YEHLIKE+araby.WAWLIKE+araby.TEHLIKE+araby.SMALL+araby.MOON+araby.SUN+araby.NAMES'''
print(araby.is_arabicrange("ۦ"))
